In [2]:
### SIREN ###
### Required Libraries ###
### PART 1 
### NOTHING CHANGED HERE!
#------------------------------------------------------------------------
from datetime import datetime
import json
from dateutil.relativedelta import relativedelta
from botocore.vendored import requests

### Functionality Helper Functions ###
def parse_int(n):
    """
    Securely converts a non-integer value to integer.
    """
    try:
        return int(n)
    except ValueError:
        return float("nan")


def build_validation_result(is_valid, violated_slot, message_content):
    """
    Define a result message structured as Lex response.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }


### Dialog Actions Helper Functions ###
def get_slots(intent_request):
    """
    Fetch all the slots and their values from the current intent.
    """
    return intent_request["currentIntent"]["slots"]


def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    """
    Defines an elicit slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "ElicitSlot",
            "intentName": intent_name,
            "slots": slots,
            "slotToElicit": slot_to_elicit,
            "message": message,
        },
    }


def delegate(session_attributes, slots):
    """
    Defines a delegate slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {"type": "Delegate", "slots": slots},
    }


def close(session_attributes, fulfillment_state, message):
    """
    Defines a close slot type response.
    """

    response = {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "Close",
            "fulfillmentState": fulfillment_state,
            "message": message,
        },
    }

    return response


#-------------------------------------------------------------------------------------
### PART 2 
### RETRIEVING PRICE OF CRYPTO FROM KRAKEN.COM!
def crypto_price_generic(ticker):
    crypto_map = {
    'xbtusd': 'XXBTZUSD',
    'adausd': 'ADAUSD',
    'ethusd': 'XETHZUSD',
    'xrpusd': 'XXRPZUSD',
    'dotusd': 'DOTUSD',   
    'filusd': 'FILUSD',
    'linkusd': 'LINKUSD',
    'ltcusd': 'XLTCZUSD', 
    }
    first_pair = ticker +"USD"
    crypto_api_url = "https://api.kraken.com/0/public/Ticker?pair="+first_pair
    xbtresponse = urllib.request.urlopen(crypto_api_url)
    xbtdata = json.load(xbtresponse)
    print("The current price of "+str(ticker)+" is: " + xbtdata['result'][crypto_map[first_pair.lower()]]['c'][0])





### FIRST SLOT TYPES - BUTTONS - WHERE THEY SHOULD LEAD?  
def coin_category(coin_type):
    coin_type = {
        "XRP": "You have selected Ripple, current price  ",
        "ETH": "You have selected Ethereum, current price  ",
        "DOT": "You have selected Polkadot, current price  ",
        "FIL": "You have selected Filecoin, current price ",
        "LINK": "You have selected Chainlink, current price  ",
        "USDT": "You have selected Tether, current price  ",
        "LTC": "You have selected Litecoin, current price  ",
        "XBT": "You have selected Bitcoin, current price  ",
        "ADA": "You have selected Cardano, current price  ",
    }    
    return coin_type[coin_type.lower()]






### PART 3 Intents Handlers ###

def Buy_Order(intent_request):
    """
    Performs dialog management and fulfillment for buying order.
    """
    buy_coin = get_slots(intent_request)["BuyCoin"]
    quantity_buy = get_slots(intent_request)["QuantityBuy"]
    buy_order_type = get_slots(intent_request)["BuyOrderType"]
    buy_order_funding = get_slots(intent_request)["BuyOrderFunding"]
    # if source == "DialogCodeHook":
        # Perform basic validation on the supplied input slots.
        # Use the elicitSlot dialog action to re-prompt
        # for the first violation detected.
        # slots = get_slots(intent_request)
        # validation_result = validate_data(buy_coin, quantity_buy, buy_order_type, buy_order_funding, buy_confirmation_total, intent_request)
        # if not validation_result["isValid"]:
            # slots[validation_result["violatedSlot"]] = None  # Cleans invalid slot
            # Returns an elicitSlot dialog to request new data for the invalid slot
            # return elicit_slot(
            #     intent_request["sessionAttributes"],
            #     intent_request["currentIntent"]["name"],
            #     slots,
            #     validation_result["violatedSlot"],
            #     validation_result["message"],
            # )
        # Fetch current session attibutes
        # output_session_attributes = intent_request["sessionAttributes"]
        # return delegate(output_session_attributes, get_slots(intent_request))
    order_price = 0
    if buy_order_funding == "ETH":
        pair = (buy_coin + buy_order_funding).lower()
        order_price = price_eth(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif buy_order_funding == "XBT":
        pair = (buy_coin + buy_order_funding).lower()
        order_price = price_xbt(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
        
    #this was added-----------------------------------------------------------
    elif buy_order_funding =="XRP":
        pair = (buy_coin + buy_order_funding).lower()
        order_price = price_xrp(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif buy_order_funding =="DOT":
        pair=(buy_coin + buy_order_funding).lower()
        order_price = price_dot(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif buy_order_funding =="FIL":
        pair = (buy_coin + buy_order_funding).lower()
        order_price = price_fil(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif buy_order_funding =="LINK":
        pair = (buy_coin + buy_order_funding).lower()
        order_price = price_link(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif buy_order_funding =="LTC":
        pair = (buy_coin + buy_order_funding).lower()
        order_price = price_ltc(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif buy_order_funding =="ADA":
        pair = (buy_coin + buy_order_funding).lower()
        order_price = price_ada(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    #----------------------------------------------------------------------------
    
    else:
        pair = (buy_coin + buy_order_funding).lower()
        order_price = price_usd(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,2)
    return close(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """Great news, your buy order has been filled for {} {} for a total amount {} {}!
            """.format(
                 quantity_buy, buy_coin, buy_order_funding, total_cost
            ),
        },
    )
    
    """ Performs dialog management and fulfillment for sell order. """
    
def Sell_Order(intent_request):
    sell_coin = get_slots(intent_request)["SellCoin"]
    quantity_sell = get_slots(intent_request)["QuantitySell"]
    sell_order_type = get_slots(intent_request)["SellOrderType"]
    sell_funding_type = get_slots(intent_request)["SellFundingType"]
    # if source == "DialogCodeHook":
        # Perform basic validation on the supplied input slots.
        # Use the elicitSlot dialog action to re-prompt
        # for the first violation detected.
        # slots = get_slots(intent_request)
        # validation_result = validate_data(buy_coin, quantity_buy, buy_order_type, buy_order_funding, buy_confirmation_total, intent_request)
        # if not validation_result["isValid"]:
            # slots[validation_result["violatedSlot"]] = None  # Cleans invalid slot
            # Returns an elicitSlot dialog to request new data for the invalid slot
            # return elicit_slot(
            #     intent_request["sessionAttributes"],
            #     intent_request["currentIntent"]["name"],
            #     slots,
            #     validation_result["violatedSlot"],
            #     validation_result["message"],
            # )
        # Fetch current session attibutes
        # output_session_attributes = intent_request["sessionAttributes"]
        # return delegate(output_session_attributes, get_slots(intent_request))
    order_price = 0
    if sell_funding_type == "ETH":
        pair = (sell_coin + sell_funding_type).lower()
        order_price = price_eth(pair)
        total_cost = round(float(quantity_sell) * float(order_price) * 1.0026,8)
    elif sell_funding_type == "XBT":
        pair = (sell_coin + sell_funding_type).lower()
        order_price = price_xbt(pair)
        total_cost = round(float(quantity_sell) * float(order_price) * 1.0026,8)
       
    
    
    
        #this was added-----------------------------------------------------------
    elif sell_funding_type =="XRP":
        pair = (sell_coin + sell_funding_type).lower()
        order_price = price_xrp(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif sell_funding_type =="DOT":
        pair = (sell_coin + sell_funding_type).lower()
        order_price = price_dot(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif sell_funding_type =="FIL":
        pair = (sell_coin + sell_funding_type).lower()
        order_price = price_fil(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif sell_funding_type =="LINK":
        pair = (sell_coin + sell_funding_type).lower()
        order_price = price_link(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif sell_funding_type =="LTC":
        pair = (sell_coin + sell_funding_type).lower()
        order_price = price_ltc(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    elif buy_order_funding =="ADA":
        pair = (sell_coin + sell_funding_type).lower()
        order_price = price_ada(pair)
        total_cost = round(float(quantity_buy) * float(order_price) * 1.0016,8)
    #----------------------------------------------------------------------------
        
   



    else:
        pair = (sell_coin + sell_funding_type).lower()
        order_price = price_usd(pair)
        total_cost = round(float(quantity_sell) * float(order_price) * 1.0026,2)
    return close(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """Great news, your sell order has been filled for {} {} for a total amount {} {}!
            """.format(
                 quantity_sell, sell_coin, sell_funding_type, total_cost
            ),
        },
    )
    
    
def trading_simplified(ticker, exchange):
    crypto_map = {
    'xbtusd': 'XXBTZUSD',
    'adausd': 'ADAUSD',
    'ethusd': 'XETHZUSD',
    'xrpusd': 'XXRPZUSD',
    'dotusd': 'DOTUSD',   
    'filusd': 'FILUSD',
    'linkusd': 'LINKUSD',
    'ltcusd': 'XLTCZUSD', 
    }
    bitcoin_ethereum_exchange = int(input("How much of "+ticker+" would you like to exchange in " + exchange+"?"))
    first_pair = ticker +"USD"
    bitcoin_api_url = "https://api.kraken.com/0/public/Ticker?pair="+first_pair
    xbtresponse = urllib.request.urlopen(bitcoin_api_url)
    xbtdata = json.load(xbtresponse)
    #print(xbtdata)
    response12 = xbtdata['result'][crypto_map[first_pair.lower()]]['c'][0]
    second_pair = exchange+"USD"
    api_conditions_url = "https://api.kraken.com/0/public/Ticker?pair="+second_pair
    ethresponse = urllib.request.urlopen(api_conditions_url)
    ethdata = json.load(ethresponse)
    response123 = ethdata['result'][crypto_map[second_pair.lower()]]['c'][0]

    exchange = float(response12) / float(response123)
    exchange = round(exchange, 4)
    return exchange * bitcoin_ethereum_exchange

def crypto_price_generic(ticker):
    crypto_map = {
    'xbtusd': 'XXBTZUSD',
    'adausd': 'ADAUSD',
    'ethusd': 'XETHZUSD',
    'xrpusd': 'XXRPZUSD',
    'dotusd': 'DOTUSD',   
    'filusd': 'FILUSD',
    'linkusd': 'LINKUSD',
    'ltcusd': 'XLTCZUSD', 
    }
    first_pair = ticker +"USD"
    crypto_api_url = "https://api.kraken.com/0/public/Ticker?pair="+first_pair
    xbtresponse = urllib.request.urlopen(crypto_api_url)
    xbtdata = json.load(xbtresponse)
    print("The current price of "+str(ticker)+" is: " + xbtdata['result'][crypto_map[first_pair.lower()]]['c'][0])

def Stake(intent_request):
    stake_coin = get_slots(intent_request)["StakeCoin"]
    stake_amount = get_slots(intent_request)["StakeAmount"]
    return close(
        intent_request["sessionAttributes"],
        "Fulfilled",
        {
            "contentType": "PlainText",
            "content": """Congratulations, you have successfully staked {} {} at 12%, payed out twice weekly!
            """.format(
                 stake_amount, stake_coin
            ),
        },
    )
    

    
### Intents Dispatcher ###
def dispatch(intent_request):
    """
    Called when the user specifies an intent for this bot.
    """

    intent_name = intent_request["currentIntent"]["name"]

    # Dispatch to bot's intent handlers
    if intent_name == "BuyOrder":
        return Buy_Order(intent_request)
    if intent_name == "SellOrder":
        return Sell_Order(intent_request)
    if intent_name == "FetchPrice":
        return crypto_price_generic(intent_request)
    if intent_name == "Stake":
        return Stake(intent_request)

    raise Exception("Intent with name " + intent_name + " not supported")


### Main Handler ###
def lambda_handler(event, context):
    """
    Route the incoming request based on intent.
    The JSON body of the request is provided in the event slot.
    """

    return dispatch(event)


